### 导入“装备知识图谱”与“十一打击群”

以下是两个辅助函数，用来存取和读取pkl文件

In [7]:
import pickle
def save_as_pickle(a_list_or_dict, save_name):
    '''保存数据结构为pkl文件'''
    save_file = open(save_name + '.pkl', 'wb')
    pickle.dump(a_list_or_dict, save_file)
    save_file.close()

def load_pickle_file(file_path):
    '''加载pkl文件'''
    a_list_or_dict = open(file_path, 'rb')
    return pickle.load(a_list_or_dict)

#### 1 连接图数据库
每次也可以通过这个模块来清空数据库

In [1]:
from py2neo import Graph, Node, Relationship, Graph, NodeMatcher, RelationshipMatcher
graph = Graph('http://localhost:7474', auth=('neo4j', 'Sicdp2021cjw'))
all_node_type = 'ontology_node' # 本体结点类型
graph.delete_all() # 清空图数据库

#### 2 构建“十一打击群”概念图谱-图结构
本体中具有“图结构”和“树结构”，这里“十一打击群”的本体图结构是从“五元组”数据中抽取而来。除此之外我们手动添加了两条规则（`海军将领`,`国籍`,`国家`）与（`州`,`隶属`,`国家`），为了在基于本体规则的补全那里做演示。   
这里将数据构造好之后顺便插入了图数据库，方便查看。

In [3]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('./triples_10_16.xlsx') # 十一打击群元组
triples = df.values.tolist()
## 插入本体中叶子结点的图结构
graph_11 = set()
for triple in triples:
    h, h_t, t, t_t, r = triple
    graph_11.add((h_t, all_node_type ,r , t_t, all_node_type))
# graph_11.add(('海军将领', all_node_type, '国籍', '国家', all_node_type))
# graph_11.add(('州', all_node_type, '隶属', '国家', all_node_type))
graph_11 = list(graph_11)
eleven_strike_swarms_graph_structure = set()
for triple in tqdm(graph_11):
    head, head_typ, rel, tail, tail_typ = triple
    node1 = Node(head_typ, name=head)
    node2 = Node(tail_typ, name=tail)
    graph.merge(node1, head_typ, "name")
    graph.merge(node2, tail_typ, "name")
    graph.create(Relationship(node1, rel, node2))
    eleven_strike_swarms_graph_structure.add((head, rel, tail))

100%|██████████| 41/41 [00:02<00:00, 15.95it/s]


#### 3 构建“十一打击群”概念图谱-树结构
这部分的数据是通过观察核心本体树，并且将现有的类别挂载到本体树上来构成的。   
同样，构造好的树结构通过图数据库来展示。

In [2]:
## thing与第一层本体类别的连接
first_layer = ['资源', '环境', '使命', '场景', '活动', '目标', '能力', '计划', '指导', '位置', '度量', '结果']
eleven_strike_swarms_tree_structure = set()
node_thing = Node(all_node_type, name='Thing')
graph.merge(node_thing , all_node_type, "name")
for node in first_layer:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_thing))
    eleven_strike_swarms_tree_structure.add((node, '属于', 'Thing'))
## 资源与第二层本体类别的连接
node_resource = Node(all_node_type, name='资源')
graph.merge(node_resource , all_node_type, "name")
layer_after_resource = ['装备物资', '执行者', '国防工程', '信息']
for node in layer_after_resource:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_resource))
    eleven_strike_swarms_tree_structure.add((node, '属于', '资源'))
## 装备物资与其子结点的连接
node_equip = Node(all_node_type, name='装备物资')
graph.merge(node_equip , all_node_type, "name")
layer_after_equip = ['火炮', '坦克装甲车辆', '枪械与单兵', '太空装备', '爆炸物', '舰船舰艇', '飞行器', '导弹武器']
for node in layer_after_equip:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_equip))
    eleven_strike_swarms_tree_structure.add((node, '属于', '装备物资'))
## 国防工程
node_natidefe = Node(all_node_type, name='国防工程')
graph.merge(node_natidefe , all_node_type, "name")
layer_after_natidefe = ['研发单位', '海军基地', '制造厂']
for node in layer_after_natidefe:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_natidefe))
    eleven_strike_swarms_tree_structure.add((node, '属于', '国防工程'))
## 执行者
node_executor = Node(all_node_type, name='执行者')
graph.merge(node_executor, all_node_type, "name")
layer_after_executor = ['海军将领', '舰载机联队', '驱逐舰中队', '作战中队']
for node in layer_after_executor:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_executor))
    eleven_strike_swarms_tree_structure.add((node, '属于', '执行者'))
## 位置
node_location = Node(all_node_type, name='位置')
graph.merge(node_location, all_node_type, "name")
node_country = Node(all_node_type, name='国家')
graph.merge(node_country, all_node_type, "name")
graph.create(Relationship(node_country, '属于', node_location))
eleven_strike_swarms_tree_structure.add(('国家', '属于', '位置'))
layer_after_country = ['州', '地区']
for node in layer_after_country:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_country))
    eleven_strike_swarms_tree_structure.add((node, '属于', '国家'))
## 舰船舰艇
node_naval_equipment = Node(all_node_type, name='舰船舰艇')
graph.merge(node_naval_equipment, all_node_type, "name")
layer_after_naval_equipment = ['航空母舰','驱逐舰','巡洋舰','补给舰','航母打击群']
for node in layer_after_naval_equipment:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_naval_equipment))
    eleven_strike_swarms_tree_structure.add((node, '属于', '舰船舰艇'))

#### 4 构建“装备知识图谱”概念图谱-图结构

In [8]:
from pprint import pprint
graph_11 = set()
openkg = load_pickle_file('equipment_kg.pkl')
for triple in openkg:
    h, h_t,r, t, t_t,  = triple
    graph_11.add((h_t, all_node_type ,r , t_t, all_node_type))
graph_11 = list(graph_11)
knowledge_graph_of_equipment_graph_structure = set()
for triple in tqdm(graph_11):
    head, head_typ, rel, tail, tail_typ = triple
    node1 = Node(head_typ, name=head)
    node2 = Node(tail_typ, name=tail)
    graph.merge(node1, head_typ, "name")
    graph.merge(node2, tail_typ, "name")
    graph.create(Relationship(node1, rel, node2))
    knowledge_graph_of_equipment_graph_structure.add((head, rel, tail))

100%|██████████| 165/165 [00:01<00:00, 98.75it/s] 


#### 5 构建“装备知识图谱”概念图谱-树结构
注意这里是将“装备知识图谱”中的类别挂到了之前在之前由“十一打击群”形成的本体上，如果想单独生成“装备知识图谱”的本体，请使用代码6。

In [6]:
import json
from tqdm import tqdm
def load_military_json():
    """加载military的json文件"""
    with open("./military_format.json",'r',encoding='utf-8') as load_f:
        load_dict = json.load(load_f)
    return load_dict
openkg = load_military_json()
knowledge_graph_of_equipment_tree_structure = set()
## 链接装备类型与装备大类
for equip_dict in tqdm(openkg):
    node1 = Node(all_node_type, name=equip_dict['类型'])
    node2 = Node(all_node_type, name=equip_dict['大类'])
    graph.merge(node1, all_node_type, "name")
    graph.merge(node2, all_node_type, "name")
    graph.create(Relationship(node1, '属于', node2))
    knowledge_graph_of_equipment_tree_structure.add((equip_dict['类型'], '属于', equip_dict['大类']))

100%|██████████| 5800/5800 [00:41<00:00, 138.87it/s]


#### 6 装备知识图谱的树结构（从thing开始）

In [10]:
import json
from tqdm import tqdm
def load_military_json():
    """加载military的json文件"""
    with open("./military_format.json",'r',encoding='utf-8') as load_f:
        load_dict = json.load(load_f)
    return load_dict
openkg = load_military_json()
## 装备物资与其子结点的连接
node_equip = Node(all_node_type, name='Thing')
graph.merge(node_equip , all_node_type, "name")
layer_after_equip = ['火炮', '坦克装甲车辆', '枪械与单兵', '太空装备', '爆炸物', '舰船舰艇', '飞行器', '导弹武器']
for node in layer_after_equip:
    node_fl = Node(all_node_type, name=node)
    graph.merge(node_fl, all_node_type, "name")
    graph.create(Relationship(node_fl, '属于', node_equip))
## 链接装备类型与装备大类
for equip_dict in tqdm(openkg):
    node1 = Node(all_node_type, name=equip_dict['类型'])
    node2 = Node(all_node_type, name=equip_dict['大类'])
    graph.merge(node1, all_node_type, "name")
    graph.merge(node2, all_node_type, "name")
    graph.create(Relationship(node1, '属于', node2))

100%|██████████| 5800/5800 [00:39<00:00, 146.14it/s]



#### 7 保存本体的图结构和树结构
如果需要则将以上生成的结构保存为pkl文件。

In [33]:
# save_as_pickle(eleven_strike_swarms_tree_structure, 'eleven_strike_swarms_tree_structure')
# save_as_pickle(eleven_strike_swarms_graph_structure, 'eleven_strike_swarms_graph_structure')
# save_as_pickle(knowledge_graph_of_equipment_tree_structure, 'knowledge_graph_of_equipment_tree_structure')
# save_as_pickle(knowledge_graph_of_equipment_graph_structure, 'knowledge_graph_of_equipment_graph_structure')
# pprint(eleven_strike_swarms_graph_structure)
# pprint(knowledge_graph_of_equipment_tree_structure)
# pprint(knowledge_graph_of_equipment_graph_structure)
# pprint(load_pickle_file('eleven_strike_swarms_tree_structure.pkl'))
# pprint(load_pickle_file('eleven_strike_swarms_graph_structure.pkl'))
# pprint(load_pickle_file('knowledge_graph_of_equipment_tree_structure.pkl'))
# pprint(load_pickle_file('knowledge_graph_of_equipment_graph_structure.pkl'))

#### 8 构建“十一打击群”图谱
“十一打击群”五元组数据

In [12]:
from py2neo import Graph, Node, Relationship, Graph, NodeMatcher, RelationshipMatcher
graph = Graph('http://localhost:7474', auth=('neo4j', 'Sicdp2021cjw'))
graph.delete_all() # 清空图数据库
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('E:/neo4j/triples_10_16.xlsx')
triples = df.values.tolist()
new_triples = []
for triple in triples:
    new_triple = []
    for str in triple:
        str = ''.join(str.split()) # 去掉空格字符
        new_triple.append(str)
    new_triples.append(new_triple)
for triple in tqdm(new_triples):
    head, head_typ, tail, tail_typ, rel = triple # 取出头实体、尾实体、关系
    head_node = Node(head_typ, name=head)
    tail_node = Node(tail_typ, name=tail)
    graph.merge(head_node, head_typ, "name")
    graph.merge(tail_node, tail_typ, "name")                  
    graph.create(Relationship(head_node, rel, tail_node))
new_triples

100%|██████████| 357/357 [00:03<00:00, 109.07it/s]


[['第十一航母打击群', '航母打击群', '尼米兹号航空母舰', '航空母舰', '航空母舰'],
 ['第十一航母打击群', '航母打击群', '第17舰载机联队', '舰载机联队', '舰载机联队'],
 ['第十一航母打击群', '航母打击群', '第9驱逐舰中队', '驱逐舰中队', '驱逐舰中队'],
 ['第十一航母打击群', '航母打击群', '埃弗里特海军基地', '海军基地', '母港'],
 ['埃弗里特海军基地', '海军基地', '华盛顿州', '州', '位置'],
 ['第17舰载机联队', '舰载机联队', '第22突击战斗机中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第94突击战斗机中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第137突击战斗机中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第323海军陆战队攻击中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第139电子攻击中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第116航母机载预警中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第6直升机海上作战中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第73直升机海上打击中队', '作战中队', '作战中队'],
 ['第17舰载机联队', '舰载机联队', '第30舰队后勤支援中队第4分队', '作战中队', '作战中队'],
 ['第9驱逐舰中队', '驱逐舰中队', '约翰保罗琼斯号导弹驱逐舰', '驱逐舰', '驱逐舰'],
 ['第9驱逐舰中队', '驱逐舰中队', '韦恩·迈耶号导弹驱逐舰', '驱逐舰', '驱逐舰'],
 ['第9驱逐舰中队', '驱逐舰中队', '斯特雷特号导弹驱逐舰', '驱逐舰', '驱逐舰'],
 ['第9驱逐舰中队', '驱逐舰中队', '拉尔夫·约翰逊号导弹驱逐舰', '驱逐舰', '驱逐舰'],
 ['第9驱逐舰中队', '驱逐舰中队', '柯蒂斯·威尔伯号导弹驱逐舰', 

#### 9 构建“十一打击群”图谱（包含规则生成的五元组）
“十一打击群”五元组数据

In [2]:
import pandas as pd
from tqdm import tqdm
def link_completion():
    res_tuples = list()
    state_set = set()
    df = pd.read_excel('E:/neo4j/triples_10_16.xlsx')
    triples = df.values.tolist()
    new_triples = []
    for triple in triples:
        new_triple = []
        for str in triple:
            str = ''.join(str.split()) # 去掉空格字符
            new_triple.append(str)
        new_triples.append(new_triple)
    for tup in new_triples:
        h, h_typ, t, t_typ, r = tup
        if (h_typ, r, t_typ) == ('海军将领', '家庭信息', '州'):
            res_tuples.append((h, h_typ,  '美国', '国家', '国籍'))
        if h_typ == '州':
            state_set.add(h)
        if t_typ == '州':
            state_set.add(t)
    for state in list(state_set):
          res_tuples.append((state, '州',  '美国', '国家', '隶属'))
    return new_triples + res_tuples
new_triples = link_completion()
for triple in tqdm(new_triples):
    head, head_typ, tail, tail_typ, rel = triple # 取出头实体、尾实体、关系
    head_node = Node(head_typ, name=head)
    tail_node = Node(tail_typ, name=tail)
    graph.merge(head_node, head_typ, "name")
    graph.merge(tail_node, tail_typ, "name")                  
    graph.create(Relationship(head_node, rel, tail_node))

100%|██████████| 447/447 [00:08<00:00, 51.91it/s]


#### 10 构建“装备知识图谱”图谱

In [14]:
from tqdm import tqdm
openkg = load_pickle_file('equipment_kg.pkl')
res = []
for triple in tqdm(openkg):
    h, h_t,r, t, t_t,  = triple # 取出头实体、尾实体、关系
    node1 = Node(h_t, name=h)
    node2 = Node(t_t, name=t)
    graph.merge(node1, h_t, "name")
    graph.merge(node2, t_t, "name")
    graph.create(Relationship(node1, rel, node2))
openkg

100%|██████████| 8281/8281 [01:03<00:00, 129.79it/s]


[('歼-16战机', '战斗机', '产国', '中国', '国家'),
 ('歼-16战机', '战斗机', '研发单位', '中国沈阳飞机公司', '研发单位'),
 ('FC-1“枭龙”/JF-17“雷电”多用途攻击机', '战斗机', '产国', '中国', '国家'),
 ('FC-1“枭龙”/JF-17“雷电”多用途攻击机', '战斗机', '研发单位', '成都飞机公司', '研发单位'),
 ('苏-27战斗机', '战斗机', '产国', '苏/俄', '国家'),
 ('苏-27战斗机', '战斗机', '研发单位', '苏霍伊设计局', '研发单位'),
 ('LCA战斗机', '战斗机', '产国', '印度', '国家'),
 ('LCA战斗机', '战斗机', '研发单位', '印度HAL公司', '研发单位'),
 ('苏-30/苏-33“侧卫” 制空战斗机和对地攻击机', '战斗机', '产国', '苏/俄', '国家'),
 ('苏-30/苏-33“侧卫” 制空战斗机和对地攻击机', '战斗机', '研发单位', '苏霍伊设计局', '研发单位'),
 ('F-16“战隼”', '战斗机', '产国', '美国', '国家'),
 ('F-16“战隼”', '战斗机', '研发单位', '通用动力公司', '研发单位'),
 ('麦克唐纳F-15E“打击鹰” 双座双发打击战斗机', '战斗机', '产国', '美国', '国家'),
 ('麦克唐纳F-15E“打击鹰” 双座双发打击战斗机', '战斗机', '研发单位', '麦克唐纳-道格拉斯公司', '研发单位'),
 ('歼-31战斗机', '战斗机', '产国', '中国', '国家'),
 ('歼-31战斗机', '战斗机', '研发单位', '沈阳飞机公司', '研发单位'),
 ('F-15鹰式战斗机', '战斗机', '产国', '美国', '国家'),
 ('F-15鹰式战斗机', '战斗机', '研发单位', '美国波音公司', '研发单位'),
 ('F-16“战隼”', '战斗机', '产国', '中国', '国家'),
 ('F-16“战隼”', '战斗机', '研发单位', '成都飞机工业集团', '研发单位'),
 ('F-35“闪电II”', '战斗机

#### 11 将数据插入数据库

In [1]:
# 导入python插入数据到数据库的函数
from pyMysql import mysql_query, mysql_in_up_re
from linkPrediction2Mysql import get_time

In [ ]:
status = '未入库'
source = '十一打击群'
sql = 'insert into candidate_triples(head, head_category, relation, tail, tail_category, status, time, source) values(%s, %s, %s, %s, %s, %s, %s, %s)'
mysql_in_up_re(sql, (head, head_typ, rel, tail, tail_typ, status, get_time(), source))